# Name : Samujjwaal Dey

## CS 582 Information Retrieval : Homework 2

## Imports and Initializations

In [1]:
# Importing dependancy libraries
import os
import pandas as pd
import numpy as np
import re
import math as m
from collections import Counter
from bs4 import BeautifulSoup
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
stop_list = stopwords.words('english')
# sorted(stop_list)

In [2]:
# Declaring variables for file path
in_path = 'cranfieldDocs'
out_path = 'preprocessed_cranfieldDocs'

# Declaring variables for query files
query = 'queries.txt'
preproc_query = 'preprocessed_queries.txt'

# Declaring variable for file with query relevance values
relevance = 'relevance.txt'

# Checking if the preprocessed docs folder exists already
if not os.path.isdir(out_path):
    os.mkdir(out_path)

# Getting all filenames from the docs folder
filenames = os.listdir(in_path)  # To generate file path
# print(filenames)

# Initiallizing Porter Stemmer object
st = PorterStemmer()

# Initializing regex to remove words with one or two characters length
shortword = re.compile(r'\W*\b\w{1,2}\b')

## Preprocessing the documents and queries file

In [3]:
def tokenize(data):
    """Preprocesses the string given as input. Converts to lower case,
    removes the punctuations and numbers, splits on whitespaces, 
    removes stopwords, performs stemming & removes words with one or 
    two characters length.

    Arguments:
        data {string} -- string to be tokenized

    Returns:
        string -- string of tokens generated
    """

    # converting to lower case
    lines = data.lower()

    # removing punctuations by using regular expression
    lines = re.sub('[^A-Za-z]+', ' ', lines)

    # splitting on whitespaces to generate tokens
    tokens = lines.split()

    # removing stop words from the tokens
    clean_tokens = [word for word in tokens if word not in stop_list]

    # stemming the tokens
    stem_tokens = [st.stem(word) for word in clean_tokens]

    # checking for stopwords again
    clean_stem_tokens = [word for word in stem_tokens if word not in stop_list]

    # converting list of tokens to string
    clean_stem_tokens = ' '.join(map(str,  clean_stem_tokens))

    # removing tokens with one or two characters length
    clean_stem_tokens = shortword.sub('', clean_stem_tokens)

    return clean_stem_tokens

In [4]:
def extractTokens(beautSoup, tag):
    """Extract tokens of the text between a specific SGML <tag>. The function
    calls tokenize() function to generate tokens from the text.

    Arguments:
        beautSoup {bs4.BeautifulSoup} -- soup bs object formed using text of a file
        tag {string} -- target SGML <tag>

    Returns:
        string -- string of tokens extracted from text between the target SGML <tag>
    """

    # extract text of a particular SGML <tag>
    textData = beautSoup.findAll(tag)

    # converting to string
    textData = ''.join(map(str, textData))
    # remove the SGML <tag> from text
    textData = textData.replace(tag, '')

    # calling function to generate tokens from text
    textData = tokenize(textData)

    return textData

In [5]:
# Preprocessing all the documents in the cranfieldDocs directory

"""This cell might take about 20 seconds to run."""

for fname in filenames:

    # generate filenames
    infilepath = in_path + '/' + fname
    outfilepath = out_path + '/' + fname

    with open(infilepath) as infile:
        with open(outfilepath, 'w') as outfile:

            # read all text in a file
            fileData = infile.read()

            # creating BeautifulSoup object to extract text between SGML tags
            soup = BeautifulSoup(fileData)

            # extract tokens for <title>
            title = extractTokens(soup, 'title')

            # extract tokens for <text>
            text = extractTokens(soup, 'text')

            # write tokens for <title> into new file
            outfile.write(title)
            outfile.write(" ")

            # write tokens for <text> into new file
            outfile.write(text)
        outfile.close()
    infile.close()

In [35]:
# Preprocessing the queries.txt file
q = open(query)

# opening new file to write preprocessed tokens into
new_q = open(preproc_query, 'w')

# read each line of file seperately
text = q.readlines()
for line in text:
    
    # if condition to avoid newline character in the end of file
    if(line != text[-1]):
        query_tokens = tokenize(line)
        new_q.write(query_tokens + '\n')
    else:
        query_tokens = tokenize(line)
        new_q.write(query_tokens)

In [36]:
# Generate a single list of all preprocessed docs
all_docs = []

for fname in filenames:
    outfilepath = out_path + '/' + fname
    with open(outfilepath) as file:
        fileData = file.read()
        
        # append file data to the list
        all_docs.append(fileData)
# print(all_docs)


In [37]:
# total number of documents is 1400
no_of_docs = len(all_docs)
no_of_docs

1400

# Task 1

## Calculating df values for each term in the vocabulary

In [38]:
# create a dictionary of key-value pairs where tokens are keys and their occurence in the corpus the value
DF = {}

for i in range(no_of_docs):
    tokens = all_docs[i].split()
    for w in tokens:
        try:
            # add token as key and doc number as value is chained
            DF[w].add(i)
        except:
            # to handle when a new token is encountered
            DF[w] = {i}

for i in DF:
    # convert to number of occurences of the token from list of documents where token occurs
    DF[i] = len(DF[i])

In [66]:
print(DF)

{'experiment': 339, 'investig': 361, 'aerodynam': 178, 'wing': 226, 'slipstream': 15, 'studi': 240, 'propel': 35, 'made': 352, 'order': 193, 'determin': 299, 'spanwis': 27, 'distribut': 360, 'lift': 158, 'increas': 206, 'due': 143, 'differ': 192, 'angl': 210, 'attack': 114, 'free': 213, 'stream': 230, 'veloc': 314, 'ratio': 296, 'result': 692, 'intend': 12, 'part': 106, 'evalu': 70, 'basi': 68, 'theoret': 235, 'treatment': 41, 'problem': 323, 'compar': 247, 'span': 40, 'load': 200, 'curv': 123, 'togeth': 40, 'support': 70, 'evid': 34, 'show': 202, 'substanti': 41, 'increment': 17, 'produc': 78, 'destal': 2, 'boundari': 467, 'layer': 411, 'control': 61, 'effect': 541, 'integr': 147, 'remain': 50, 'subtract': 2, 'found': 322, 'agre': 59, 'well': 163, 'potenti': 58, 'flow': 729, 'theori': 455, 'empir': 32, 'specif': 63, 'configur': 81, 'experi': 152, 'simpl': 184, 'shear': 99, 'past': 84, 'flat': 187, 'plate': 227, 'incompress': 132, 'fluid': 160, 'small': 239, 'viscos': 63, 'high': 233, 

In [40]:
# count number of unique words in the corpus
vocab_size = len(DF)
print(vocab_size)

4394


In [41]:
# create vocabulary list of all unique words
vocab = [term for term in DF]
print(vocab)

['experiment', 'investig', 'aerodynam', 'wing', 'slipstream', 'studi', 'propel', 'made', 'order', 'determin', 'spanwis', 'distribut', 'lift', 'increas', 'due', 'differ', 'angl', 'attack', 'free', 'stream', 'veloc', 'ratio', 'result', 'intend', 'part', 'evalu', 'basi', 'theoret', 'treatment', 'problem', 'compar', 'span', 'load', 'curv', 'togeth', 'support', 'evid', 'show', 'substanti', 'increment', 'produc', 'destal', 'boundari', 'layer', 'control', 'effect', 'integr', 'remain', 'subtract', 'found', 'agre', 'well', 'potenti', 'flow', 'theori', 'empir', 'specif', 'configur', 'experi', 'simpl', 'shear', 'past', 'flat', 'plate', 'incompress', 'fluid', 'small', 'viscos', 'high', 'speed', 'viscou', 'two', 'dimension', 'bodi', 'usual', 'necessari', 'consid', 'shock', 'wave', 'emit', 'nose', 'lead', 'edg', 'consequ', 'exist', 'inviscid', 'rotat', 'region', 'situat', 'aris', 'instanc', 'hyperson', 'somewhat', 'prandtl', 'classic', 'origin', 'outsid', 'irrot', 'must', 'possibl', 'vortic', 'recen

## Calculating tf-idf values for each term in the vocabulary

In [42]:
doc = 0

# creating dictionary to store tf-idf values for each term in the vocabulary
tf_idf = {}

for i in range(no_of_docs):
    
    tokens = all_docs[i].split()
    
    # counter object to efficiently count number of occurence of a term in a particular document
    counter = Counter(tokens)
    words_count = len(tokens)
    
    for token in np.unique(tokens):
        
        # counting occurence of term in object using counter object
        tf = counter[token]/words_count
        # retrieving df values from DF dictionary
        df = DF[token] if token in vocab else 0
        
        # adding 1 to numerator & denominator to avoid divide by 0 error
        idf = np.log((no_of_docs+1)/(df+1))
        
        tf_idf[doc, token] = tf*idf

    doc += 1

In [43]:
# print(tf_idf)
tf_idf

{(0, 'aerodynam'): 0.02604500937337028,
 (0, 'agre'): 0.03988097448246716,
 (0, 'angl'): 0.023963081175454943,
 (0, 'attack'): 0.03164568883511085,
 (0, 'basi'): 0.03811183597138921,
 (0, 'boundari'): 0.013879408233156457,
 (0, 'compar'): 0.021917883546481325,
 (0, 'configur'): 0.035926864545224736,
 (0, 'control'): 0.03946591343407488,
 (0, 'curv'): 0.0306918984902781,
 (0, 'destal'): 0.23340490851907206,
 (0, 'determin'): 0.01950834267950387,
 (0, 'differ'): 0.07527536800375145,
 (0, 'distribut'): 0.01716536187346995,
 (0, 'due'): 0.057598183462303956,
 (0, 'effect'): 0.02404241885816103,
 (0, 'empir'): 0.0474485314540573,
 (0, 'evalu'): 0.07550029542520738,
 (0, 'evid'): 0.04670371499807081,
 (0, 'experi'): 0.028031691454994574,
 (0, 'experiment'): 0.03584799819561518,
 (0, 'flow'): 0.00825186091385532,
 (0, 'found'): 0.018573281305244944,
 (0, 'free'): 0.023784373814115894,
 (0, 'increas'): 0.024205351304704276,
 (0, 'increment'): 0.11024227312508461,
 (0, 'integr'): 0.028452269273

## Forming document vectors using the tf-idf values

In [44]:
# initializing empty vector of vocabulary size
D = np.zeros((no_of_docs, vocab_size))

# creating vector of tf-idf values
for i in tf_idf:
    ind = vocab.index(i[1])
    D[i[0]][ind] = tf_idf[i]

In [45]:
# len(D)
print(D)

[[0.035848   0.01713035 0.02604501 ... 0.         0.         0.        ]
 [0.         0.01166636 0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.14559543]
 [0.         0.02182738 0.         ... 0.         0.         0.        ]]


In [46]:
def gen_vector(tokens):
    """To create a vector (with repsect to the vocabulary) of the tokens passed as input
    
    Arguments:
        tokens {list} -- list of tokens to be converted
    
    Returns:
        numpy.ndarray -- vector of tokens
    """
    Q = np.zeros((len(vocab)))
    
    counter = Counter(tokens)
    words_count = len(tokens)

    query_weights = {}
    
    for token in np.unique(tokens):
        
        tf = counter[token]/words_count
        df = DF[token] if token in vocab else 0
        idf = m.log((no_of_docs+1)/(df+1))

        try:
            ind = vocab.index(token)
            Q[ind] = tf*idf
        except:
            pass
    return Q

# Task 2

## Calculating cosine similarity between each query vectors and document vectors

In [47]:
def cosine_sim(x, y):
    """To calculate cosine similarity between 2 vectors.
    
    Arguments:
        x {numpy.ndarray} -- vector 1
        y {numpy.ndarray} -- vector 2
    
    Returns:
        numpy.float64 -- cosine similarity between vector 1 & vector 2
    """
    cos_sim = np.dot(x, y)/(np.linalg.norm(x)*np.linalg.norm(y))
    
    return cos_sim

In [48]:
def cosine_similarity(k, query):
    """To determine a ranked list of top k documents in descending order of their
    cosine similarity with the query
    
    Arguments:
        k {integer} -- top k documents to retrieve from 
        query {string} -- query whose cosine similarity is to be computed with the corpus
    
    Returns:
        numpy.ndarray -- list of top k cosine similarities between query and corpus of documents
    """

    tokens = query.split()
      
    d_cosines = []
    
    # vectorize the input query tokens
    query_vector = gen_vector(tokens)
    
    for d in D:
        d_cosines.append(cosine_sim(query_vector, d))
        
    if k == 0:
        # k=0 to retrieve all documents in descending order
        out = np.array(d_cosines).argsort()[::-1]
        
    else:
        # to retrieve the top k documents in descending order    
        out = np.array(d_cosines).argsort()[-k:][::-1]
    
    return out

In [49]:
# list of all queries from preprocessed queries file
query_file = open(preproc_query, 'r')
queries = query_file.readlines()
queries

['investig made wave system creat static pressur distribut liquid surfac\n',
 'anyon investig effect shock gener vortic heat transfer blunt bodi\n',
 'heat transfer blunt bodi absenc vortic\n',
 'gener effect flow field reynold number small\n',
 'find calcul procedur applic incompress laminar boundari layer flow problem good accuraci reason comput time\n',
 'paper applic problem calcul procedur laminar incompress flow arbitrari pressur gradient\n',
 'anyon investig shear buckl stiffen plate\n',
 'paper shear buckl unstiffen rectangular plate shear\n',
 'practic close realiti assumpt flow hyperson shock tube use nitrogen non viscou thermodynam equilibrium\n',
 'design factor use control lift drag ratio mach number']

In [50]:
def list_of_docs(k):
    """To generate a ranked list of k top documents in descending order of their cosine similarity 
    calculated against the queries. Output is a list of (query id, document id) pairs.
    
    If k=0 is given as input then list of all documnets in descending order is returned.
    
    Arguments:
        k {integer} -- number of top documents to be retrieved
    
    Returns:
        list -- list of documents in descending order of their cosine similarity
    """
    cos_sims = []
    for i in range(len(queries)):
        cs = [i, cosine_similarity(k, queries[i])]
        cos_sims.append(cs)
        
    return cos_sims


In [51]:
#to get list of all documents
no_of_top=0
list_of_docs(no_of_top)

[[0, array([ 763,  957,  406, ...,  612, 1059,  222], dtype=int64)],
 [1, array([ 323, 1394,  322, ...,  879,  878,  937], dtype=int64)],
 [2, array([323, 322, 628, ..., 818, 817,   0], dtype=int64)],
 [3, array([1077, 1221,  667, ...,  881,  262,  910], dtype=int64)],
 [4, array([ 737,    3,  381, ...,  399, 1056,  912], dtype=int64)],
 [5, array([1365,    2, 1385, ...,  881,  882,  870], dtype=int64)],
 [6, array([1399, 1397, 1129, ...,  584,  583, 1196], dtype=int64)],
 [7, array([ 399, 1398, 1397, ...,  777,  776,    0], dtype=int64)],
 [8, array([1311, 1285,  316, ...,  762,  760, 1399], dtype=int64)],
 [9, array([1379, 1123, 1187, ...,  663, 1357,  652], dtype=int64)]]

## Calculating precision and recall values for each query

In [52]:
# retrieving relevance values from relevance.txt
colnames=['query', 'relevance'] 
rel = pd.read_csv(relevance, delim_whitespace=True, names=colnames, header=None)
rel.head(10)

,query,relevance
0,1,156
1,2,666
2,2,667
3,2,1258
4,2,1394
5,2,668
6,2,670
7,2,1204
8,2,1391
9,2,1395


In [53]:
# list of relevant document numbers for a document
rel_list = []

# list of list of relevant document numbers for all documents
query_rel = []
for i in range(1,11):
    rel_list = rel[rel['query']==i]['relevance'].to_list()
    
    # append list rel_list to list of list query_list
    query_rel.append(rel_list)
print(query_rel)    

[[156], [666, 667, 1258, 1394, 668, 670, 1204, 1391, 1395, 1300, 37, 559, 630, 1107, 1213], [24, 101, 666, 667, 93, 1258, 1393, 559, 630, 662, 1104, 1107, 1204, 1213, 1300], [1391, 666, 667, 1258, 1078, 1080, 1081, 1394, 1395, 1214, 1198, 1204, 1300, 559, 630, 662, 1107, 1213], [1383, 1385, 155, 241, 1382, 1370, 1386, 111, 1384, 150, 292, 458, 479, 977, 376, 459, 1365, 62, 1366], [155, 1383, 1385, 1382, 62, 292, 241, 1370, 1384, 458, 459, 461, 1386, 1365, 1366, 111, 150, 479], [400, 419, 1387, 412, 1392, 1398, 1397, 1400, 1399], [400, 1387, 1392, 1398], [656, 1313, 1317, 1316, 1318, 1319, 1157, 1274], [1379, 1305, 1304, 40, 293, 1309, 161, 421, 1377, 1378, 1381, 225, 1380, 448, 449, 1124, 1280, 433, 923, 924, 1062, 1074, 1075, 1213]]


In [54]:
def intersection(lst1, lst2): 
    """To count number of common items between 2 lists
    
    Arguments:
        lst1 {list} -- list 1
        lst2 {list} -- list 2
    
    Returns:
        integer -- number of common items between list 1 & list 2 
    """
    lst3 = [value for value in lst1 if value in lst2] 
    return len(lst3) 

In [60]:
top = [10, 50, 100, 500]

# for top 100 docs
no_of_top = top[2]
no_of_top

100

In [58]:
def calculate_recall(k):
    """To generate list of recall values for each query for given value of k
    
    Arguments:
        k {integer} -- number of top documents to be retrieved 
    
    Returns:
        list -- list of recall values for each query
    """
    
    recall = []
    for i in range(len(queries)):
        
        # Number of relevant documents retrieved
        a = intersection(list_of_docs(k)[i][1].tolist(), query_rel[i])
        
        # Total number of relevant documents
        b = len(query_rel[i])
        r = a / b
        recall.append(r)
    return recall   
# for top 100 docs
calculate_recall(no_of_top)


[0.0,
 0.13333333333333333,
 0.2,
 0.2777777777777778,
 0.3684210526315789,
 0.2222222222222222,
 0.3333333333333333,
 0.25,
 0.5,
 0.16666666666666666]

In [59]:
np.mean(calculate_recall(no_of_top))

0.2451754385964912

In [61]:
def calculate_precision(k):
    """To generate list of precision values for each query for given value of k
    
    Arguments:
        k {[type]} -- number of top documents to be retrieved
    
    Returns:
        list -- list of precision values for each query
    """
    precision = []
    for i in range(len(queries)):
        
        # Number of relevant documents retrieved
        a = intersection(list_of_docs(k)[i][1].tolist(), query_rel[i])
        
        # Total number of documents retrieved
        b = k
        p = a / b
        precision.append(p)
    return precision

# for top 100 docs
calculate_precision(no_of_top)


[0.0, 0.02, 0.03, 0.05, 0.07, 0.04, 0.03, 0.01, 0.04, 0.04]

In [62]:
np.mean(calculate_precision(no_of_top))

0.032999999999999995

In [63]:
# To print the output of answers to questions in Task 2
# Output saved to output.txt file

"""This cell might take about 40 seconds to run."""

for t in top:
    p=0
    r=0
    print("Top {0} documents in the rank list".format(t))
    p = calculate_precision(t)
    r = calculate_recall(t)
    for i in range(len(queries)):
        print("Query: {0} \t Pr: {1} \t Re: {2}\n".format(i+1,p[i],r[i]))
    print("Avg Precision: {0}\n".format(np.mean(p)))
    print("Avg Recall: {0}\n\n".format(np.mean(r)))
    


Top 10 documents in the rank list
Query: 1 	 Pr: 0.0 	 Re: 0.0

Query: 2 	 Pr: 0.1 	 Re: 0.06666666666666667

Query: 3 	 Pr: 0.0 	 Re: 0.0

Query: 4 	 Pr: 0.1 	 Re: 0.05555555555555555

Query: 5 	 Pr: 0.1 	 Re: 0.05263157894736842

Query: 6 	 Pr: 0.2 	 Re: 0.1111111111111111

Query: 7 	 Pr: 0.3 	 Re: 0.3333333333333333

Query: 8 	 Pr: 0.1 	 Re: 0.25

Query: 9 	 Pr: 0.1 	 Re: 0.125

Query: 10 	 Pr: 0.2 	 Re: 0.08333333333333333

Avg Precision: 0.12000000000000002

Avg Recall: 0.10776315789473682


Top 50 documents in the rank list
Query: 1 	 Pr: 0.0 	 Re: 0.0

Query: 2 	 Pr: 0.02 	 Re: 0.06666666666666667

Query: 3 	 Pr: 0.02 	 Re: 0.06666666666666667

Query: 4 	 Pr: 0.04 	 Re: 0.1111111111111111

Query: 5 	 Pr: 0.06 	 Re: 0.15789473684210525

Query: 6 	 Pr: 0.06 	 Re: 0.16666666666666666

Query: 7 	 Pr: 0.06 	 Re: 0.3333333333333333

Query: 8 	 Pr: 0.02 	 Re: 0.25

Query: 9 	 Pr: 0.04 	 Re: 0.25

Query: 10 	 Pr: 0.04 	 Re: 0.08333333333333333

Avg Precision: 0.036

Avg Recall: 0.148567